<a href="https://colab.research.google.com/github/kdlogan19/CAP6610/blob/master/HW2_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# Lease Square Loss



---



In [0]:
import pandas as pd

data = pd.read_csv("winequality-red.csv", sep = ';')

In [204]:
data

fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  quality
0               7.4             0.700         0.00  ...       0.56      9.4        5
1               7.8             0.880         0.00  ...       0.68      9.8        5
2               7.8             0.760         0.04  ...       0.65      9.8        5
3              11.2             0.280         0.56  ...       0.58      9.8        6
4               7.4             0.700         0.00  ...       0.56      9.4        5
...             ...               ...          ...  ...        ...      ...      ...
1594            6.2             0.600         0.08  ...       0.58     10.5        5
1595            5.9             0.550         0.10  ...       0.76     11.2        6
1596            6.3             0.510         0.13  ...       0.75     11.0        6
1597            5.9             0.645         0.12  ...       0.71     10.2        5
1598            6.0             0.310         0.47  ...       0.66     11.0        6

[1599 rows x 12 columns]

In [0]:
wine_train = data.iloc[:1400,:11]
wine_test = data.iloc[1400:,:11]

In [0]:
y_train = data.iloc[:1400,11]
y_test = data.iloc[1400:,11]

In [218]:
import numpy as np
ones = np.ones((1400,1))
ones
ones.shape

(1400, 1)

In [219]:
A = np.block([[ones, wine_train]])
A

array([[ 1.  ,  7.4 ,  0.7 , ...,  3.51,  0.56,  9.4 ],
       [ 1.  ,  7.8 ,  0.88, ...,  3.2 ,  0.68,  9.8 ],
       [ 1.  ,  7.8 ,  0.76, ...,  3.26,  0.65,  9.8 ],
       ...,
       [ 1.  ,  7.3 ,  0.59, ...,  3.28,  0.52,  9.9 ],
       [ 1.  ,  7.  ,  0.6 , ...,  3.52,  0.62, 10.2 ],
       [ 1.  ,  7.2 ,  0.67, ...,  3.42,  0.72, 11.1 ]])

In [220]:
A.shape


(1400, 12)

In [221]:
type(y_train)

pandas.core.series.Series

In [0]:
y_train = y_train.to_numpy()

In [0]:
y_train = y_train.reshape(1400,1)

In [224]:
y_train.shape

(1400, 1)

In [0]:
res = np.matmul(np.linalg.pinv(A),y_train)

In [226]:
res

array([[ 1.87097067e+01],
       [ 2.32987600e-02],
       [-1.07620505e+00],
       [-1.45281727e-01],
       [ 6.56905767e-03],
       [-1.82541943e+00],
       [ 3.41549083e-03],
       [-3.42757516e-03],
       [-1.49504458e+01],
       [-2.91895948e-01],
       [ 8.73197347e-01],
       [ 2.77656332e-01]])

In [227]:
wine_test

fixed acidity  volatile acidity  citric acid  ...    pH  sulphates  alcohol
1400            7.9             0.690         0.21  ...  3.25       0.51      9.9
1401            7.9             0.690         0.21  ...  3.25       0.51      9.9
1402            7.6             0.300         0.42  ...  3.44       0.82     11.9
1403            7.2             0.330         0.33  ...  3.23       1.10     10.0
1404            8.0             0.500         0.39  ...  3.43       0.62     10.7
...             ...               ...          ...  ...   ...        ...      ...
1594            6.2             0.600         0.08  ...  3.45       0.58     10.5
1595            5.9             0.550         0.10  ...  3.52       0.76     11.2
1596            6.3             0.510         0.13  ...  3.42       0.75     11.0
1597            5.9             0.645         0.12  ...  3.57       0.71     10.2
1598            6.0             0.310         0.47  ...  3.39       0.66     11.0

[199 rows x 11 columns]

In [0]:
A_test = np.block([np.ones((199,1)),wine_test])

In [229]:
A_test.shape

(199, 12)

In [0]:
y_test = y_test.to_numpy()

In [0]:
y_test.reshape(199,1)
result = np.matmul(A_test,res)

In [0]:
result = np.round(result)

In [233]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,result)

0.4371859296482412







---


# *********************HUBER LOSS FUNCTION***************************************** 



---





In [0]:
wine_train
wine_train = wine_train.to_numpy()


In [0]:
import cvxpy as cp
w = cp.Variable((11,1)); b = cp.Variable()

In [0]:
obj_train = cp.sum(cp.huber(wine_train@w + b - y_train))

In [286]:
cvxpy.Problem(cvxpy.Minimize(obj_train), []).solve()

535.4177751956599

In [287]:
w.value

array([[ 5.01133982e-02],
       [-1.03205164e+00],
       [-1.83368522e-01],
       [ 2.37555652e-02],
       [-1.53504304e+00],
       [ 3.13824283e-03],
       [-3.42146282e-03],
       [-3.46798683e+01],
       [-1.00830934e-01],
       [ 8.91948016e-01],
       [ 2.63729352e-01]])

In [288]:
b.value

array(37.58293877)

In [0]:
wine_test = wine_test.to_numpy()
y_test = y_test.reshape(199,1)

In [0]:
obj_test = wine_test@w + b

In [309]:
obj_test.shape

(199, 1)

In [313]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,obj_test.value)

0.5327156477588904



---



---


# **       *********** HINGE LOSS******************    **

---



In [0]:
# Form SVM with L1 regularization problem.
import cvxpy as cp
beta = cp.Variable((n,1))
v = cp.Variable()
loss = cp.sum(cp.pos(1 - cp.multiply(Y, X @ beta - v)))
reg = cp.norm(beta, 1)
lambd = cp.Parameter(nonneg=True)
prob = cp.Problem(cp.Minimize(loss/m + lambd*reg))

In [0]:
wine_train = wine_train.to_numpy()
y_train = y_train.to_numpy()
y_train = y_train.reshape(1400,1)
wine_test = wine_test.to_numpy()
y_test = y_test.to_numpy()
y_test = y_test.reshape(199,1)


In [0]:
w2 = cp.Variable((11,1))
b2 = cp.Variable()

In [0]:
loss_train = cp.sum(cp.max(cp.abs(wine_train@w2 + b2 - y_train) - 0.5))

In [420]:
loss_train

Expression(CONVEX, UNKNOWN, ())

In [424]:
cp.Problem(cp.Minimize(loss_train),[]).solve(solver = cp.ECOS)

1.4427196879472994

In [425]:
w2.value

array([[-1.02443588e-01],
       [-1.14031934e+00],
       [-8.03873920e-01],
       [-1.02267158e-01],
       [-2.19223559e+00],
       [-6.56619772e-03],
       [ 2.54697174e-04],
       [ 3.58110026e+01],
       [-2.66588926e+00],
       [ 4.24335853e-01],
       [ 3.75420175e-01]])

In [426]:
b2.value

array(-23.29192227)

In [0]:
hinge_pred = cp.pos(wine_test@w2 + b2 - 0.5)

In [428]:
print(type(hinge_pred.value))
type(y_test)

<class 'numpy.ndarray'>


numpy.ndarray

In [429]:
mean_absolute_error(y_test,hinge_pred.value)

0.7238964232953974